# BinaryStory
This notebook will allow us to create "on the fly" van den Heuval diagrams for binary population synthesis code POSYDON. It is important to not that the visualization itself begins on code block 6. Before that point, this notebook is an example of running a pop synth model using POSYDON. 




In [ ]:
import posydon
from posydon.popsyn.binarypopulation import BinaryPopulation
from posydon.binary_evol.simulationproperties import SimulationProperties
from posydon.binary_evol.flow_chart import flow_chart
from posydon.binary_evol.CE.step_CEE import StepCEE
from posydon.binary_evol.SN.step_SN import StepSN
from posydon.binary_evol.step_end import step_end
from posydon.binary_evol.MESA.step_mesa import CO_HeMS_step, MS_MS_step, CO_HMS_RLO_step
from posydon.binary_evol.DT.step_detached import detached_step
from posydon.binary_evol.DT.double_CO import DoubleCO
from posydon.binary_evol.simulationproperties import TimingHooks, StepNamesHooks

In [ ]:
#Required imports for stiching
%matplotlib inline
import os
import pandas as pd
from PIL import Image
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import ConnectionPatch
import matplotlib.patches as mpatches
import bokeh
import cv2
import os,sys
import colorsys

In [ ]:
sim_kwargs = dict(
    flow = (flow_chart, {}),
    step_HMS_HMS = (MS_MS_step, {}),
    step_CO_HeMS = (CO_HeMS_step, {}),
    step_CO_HMS_RLO = (CO_HMS_RLO_step, {}),
    step_detached = (detached_step, {}),
    step_CE = (StepCEE, {}),
    step_SN = (StepSN, {}),
    step_dco = (DoubleCO, {}),
    step_end = (step_end, {}),
    extra_hooks = [(TimingHooks, {}), (StepNamesHooks, {})]
)

kwargs = dict(
    include_S1=True , # True, False
    S1_kwargs=dict(only_select_columns=[
                                        'state',
                                        #'metallicity',
                                        'mass',
                                        'log_R',
                                        'log_L',
                                        'lg_mdot',
                                        #'lg_system_mdot',
                                        #'lg_wind_mdot',
                                        'he_core_mass',
                                        'he_core_radius',
                                        #'c_core_mass',
                                        #'c_core_radius',
                                        #'o_core_mass',
                                        #'o_core_radius',
                                        'co_core_mass',
                                        'co_core_radius',
                                        'center_h1',
                                        'center_he4',
                                        #'center_c12',
                                        #'center_n14',
                                        #'center_o16',
                                        'surface_h1',
                                        'surface_he4',
                                        #'surface_c12',
                                        #'surface_n14',
                                        #'surface_o16',
                                        #'log_LH',
                                        #'log_LHe',
                                        #'log_LZ',
                                        #'log_Lnuc',
                                        #'c12_c12',
                                        #'center_gamma',
                                        #'avg_c_in_c_core',
                                        #'surf_avg_omega',
                                        'surf_avg_omega_div_omega_crit',
                                        #'total_moment_of_inertia',
                                        #'log_total_angular_momentum',
                                        'spin',
                                        #'conv_env_top_mass',
                                        #'conv_env_bot_mass',
                                        #'conv_env_top_radius',
                                        #'conv_env_bot_radius',
                                        #'conv_env_turnover_time_g',
                                        #'conv_env_turnover_time_l_b',
                                        #'conv_env_turnover_time_l_t',
                                        #'envelope_binding_energy',
                                        #'mass_conv_reg_fortides',
                                        #'thickness_conv_reg_fortides',
                                        #'radius_conv_reg_fortides',
                                        #'lambda_CE_1cent',
                                        #'lambda_CE_10cent',
                                        #'lambda_CE_30cent',
                                        #'lambda_CE_pure_He_star_10cent',
                                        #'profile',
                                        ]),

        optimize_ram= False,
        number_of_binaries = 1000)

kwargs2 = dict(
    extra_columns=['step_names','step_times']
)
gen_info_columns = ['state',
                   'step_names',
                    'event',
                    'step_times',
                    'separation',
                    'orbital_period',
                    'eccentricity',
                    'rl_relative_overflow_1',
                    'rl_relative_overflow_2',
                    'lg_mtransfer_rate',
                    'mass_transfer_case',
                    'trap_radius',
                    'acc_radius',
                    't_sync_rad_1',
                    't_sync_conv_1',
                    't_sync_rad_2',
                    't_sync_conv_2',
                    'time'
                   ]
state_info_columns = ['binary_index', 'state','step_names','step_times',
                      'event','S1_state','S1_mass','S2_state','S2_mass',
                      'separation','orbital_period', 'eccentricity', 'time',
                      'S1_log_R', 'S1_log_L', 'S2_log_R','S2_log_L']

In [ ]:
BinarySimulation= SimulationProperties(**sim_kwargs)
pop = BinaryPopulation(population_properties=BinarySimulation, **kwargs)
pop.evolve(breakdown_to_df=False, tqdm=True)

In [ ]:
DF = pop.to_df(**kwargs2)
DF = DF.rename_axis('binary_index').reset_index()
#bs_idx = df['binary_index'].unique()
print(DF.columns.tolist())

In [ ]:
#Visualization View Gives me the Necessary information to make the visualizations 
VisualizationView=DF[state_info_columns]
VisualizationView

In [ ]:
stellartemps = pd.read_csv('Visualization_Data/Stellar_Temperature/Teff_RGB_Hex.csv')
stellartemps


In [ ]:
#Iterator Function Getstar
#This function retrieves a specific index(star in the population) from the mother DataFrame returned from the population synthesis and 
#creates a minidataframe that has information based on the specific index.

import pandas as pd
#df = DF.rename_axis('binary_index').reset_index() #always comment out after you run so you don't create another column
def getstar(indx): 
    data = DF.loc[DF['binary_index']==indx]
    return pd.DataFrame(data)
minidataframe = getstar(90) #99 is interesting 
MiniVizView=minidataframe[state_info_columns]
MiniVizView

In [1]:
#Temperature Calculator 

Luminosity1=np.array([0])
Radius1=np.array([0])
Temperature_array=[]

def Temperature(Luminosity, Radius):
    Luminosity=10**Luminosity*3.85e33
    print(Luminosity)#W-.erg*s^-1
    rad=10**Radius*6.957e10 #solarradii->cm
    print(rad)
    sig=5.6705e-5 #stephen boltzmann constant
    A=4*np.pi*(rad)**2
    Temperature=(Luminosity/(sig*A))**(1/4)
    Temperature_array.append(Temperature)
    return Temperature
Temperature(Luminosity1,Radius1)
print('K:',Temperature_array)

NameError: name 'np' is not defined

In [ ]:
#!!!
from matplotlib.markers import MarkerStyle
import mplcursors

def chartobject(ax, img, fontsize=12): 
                ax.imshow(img)
                ax.locator_params(nbins=3)
#def temperaturefilter(S1_mass,S2_mass)
            
def getimage(dataframe): 
    plot_array=[]
    step_array=[]
    step_times=[] 
    temp_arrayS1=[]
    temp_arrayS2=[]
    mass_arrayS1=[]
    mass_arrayS2=[]
    separation_array=[]
    # Iterate over the indices in the DataFrame
    for n,index in enumerate(dataframe.index):
        # Get the filename based on the column
        filename = f"{dataframe['state'][index]}"+"_"+ f"{dataframe['step_names'][index]}"+"_"+ f"{dataframe['event'][index]}"+"_"+ f"{dataframe['S1_state'][index]}"+"_"+ f"{dataframe['S2_state'][index]}.jpg"  
        #print(filename)
        print(index)
        print(dataframe['event'][index])
        print(type(dataframe['event'][index]))
        print('timecolumn:',dataframe['time'][index])
        print(type(dataframe['time'][index]))
        if pd.isnull(dataframe['time'][index]):# if the step times column is NaN then it will just write the last column's info 
            step_times.append(step_times[n-1])  
        else: 
            step_times.append(f"{dataframe['time'][index]/1e6:.0f} Myrs")
                              
        if dataframe['event'][index]=='ZAMS': #if the functions reads ZAMS (event)
            starstate=f"{dataframe['S1_state'][index]}" + "_" + f"{dataframe['S2_state'][index]}"
            plot_array.append(starstate+'_'+"ZAMS.jpg") 
            step_array.append(f"{dataframe['step_names'][index]}")
            #step_times.append(f"{dataframe['time'][index]/1e6:.0f} myrs")
            temp_arrayS1.append(Temperature(dataframe['S1_log_L'][index],dataframe['S1_log_R'][index]))
            temp_arrayS2.append(Temperature(dataframe['S2_log_L'][index],dataframe['S2_log_R'][index]))  
            mass_arrayS1.append(f"{dataframe['S1_mass'][index]:.2f} Solar masses")
            mass_arrayS2.append(f"{dataframe['S2_mass'][index]:.2f} Solar masses")
            separation_array.append(f"{dataframe['separation'][index]:.2f} Solar Radii")
            print('Star1:', dataframe['S1_log_L'][index],dataframe['S1_log_R'][index],
                              'Star2:', dataframe['S2_log_L'][index],dataframe['S2_log_R'][index])
        
        if dataframe['state'][index]=='RLOF': #if the functions reads RLOF (state)
            starstate=f"{dataframe['S1_state'][index]}" + "_" + f"{dataframe['S2_state'][index]}"
            plot_array.append(starstate+'_'+"RLOF.jpg")
            step_array.append(f"{dataframe['step_names'][index]}")
            #step_times.append(f"{dataframe['time'][index]/1e6:.0f} myrs")
            temp_arrayS1.append(Temperature(dataframe['S1_log_L'][index],dataframe['S1_log_R'][index]))
            temp_arrayS2.append(Temperature(dataframe['S2_log_L'][index],dataframe['S2_log_R'][index]))
            mass_arrayS1.append(f"{dataframe['S1_mass'][index]:.2f} Solar masses")
            mass_arrayS2.append(f"{dataframe['S2_mass'][index]:.2f} Solar masses")
            separation_array.append(f"{dataframe['separation'][index]:.2f} Solar Radii")
            print('Star1:', dataframe['S1_log_L'][index],dataframe['S1_log_R'][index],
                              'Star2:', dataframe['S2_log_L'][index],dataframe['S2_log_R'][index])
       
        if dataframe['state'][index]=='RLO1': #if the function reads RLO1 (state)
            starstate=f"{dataframe['S1_state'][index]}" + "_" + f"{dataframe['S2_state'][index]}"
            plot_array.append(starstate+'_'+"RLO1.jpg")
            step_array.append(f"{dataframe['step_names'][index]}")
           # step_times.append(f"{dataframe['time'][index]/1e6:.0f} myrs")
            temp_arrayS1.append(Temperature(dataframe['S1_log_L'][index],dataframe['S1_log_R'][index]))
            temp_arrayS2.append(Temperature(dataframe['S2_log_L'][index],dataframe['S2_log_R'][index]))
            mass_arrayS1.append(f"{dataframe['S1_mass'][index]:.2f} Solar masses")
            mass_arrayS2.append(f"{dataframe['S2_mass'][index]:.2f} Solar masses")
            separation_array.append(f"{dataframe['separation'][index]:.2f} Solar Radii")
            print('Star1:', dataframe['S1_log_L'][index],dataframe['S1_log_R'][index],
                              'Star2:', dataframe['S2_log_L'][index],dataframe['S2_log_R'][index])
        
        if dataframe['event'][index]=='oCE1': #if the function reads oCEI (event)
            starstate=f"{dataframe['S1_state'][index]}" + "_" + f"{dataframe['S2_state'][index]}"
            plot_array.append(starstate+'_'+"oCE1.jpg")
            step_array.append(f"{dataframe['step_names'][index]}")
            #step_times.append(f"{dataframe['time'][index]/1e6:.0f} myrs")
            temp_arrayS1.append(Temperature(dataframe['S1_log_L'][index],dataframe['S1_log_R'][index]))
            temp_arrayS2.append(Temperature(dataframe['S2_log_L'][index],dataframe['S2_log_R'][index]))
            mass_arrayS1.append(f"{dataframe['S1_mass'][index]:.2f} Solar masses")
            mass_arrayS2.append(f"{dataframe['S2_mass'][index]:.2f} Solar masses")
            separation_array.append(f"{dataframe['separation'][index]:.2f} Solar Radii")
            print('Star1:', dataframe['S1_log_L'][index],dataframe['S1_log_R'][index],
                              'Star2:', dataframe['S2_log_L'][index],dataframe['S2_log_R'][index])
        
        if dataframe['event'][index]=='END':#this one is the merged image
            string='End'
            step_times.append(f"{dataframe['time'][index]}")
            if dataframe['state'][index]=='merged':
                starstate=f"{dataframe['S1_state'][index]}" + "_" + f"{dataframe['S2_state'][index]}"
                plot_array.append(starstate+string+'_'+"Merged.jpg") #format: star1_star2_End_Merged.jpg
                #step_times.append(f"{dataframe['time'][index]}")
                temp_arrayS1.append(Temperature(dataframe['S1_log_L'][index],dataframe['S1_log_R'][index]))
                temp_arrayS2.append(Temperature(dataframe['S2_log_L'][index],dataframe['S2_log_R'][index]))
                mass_arrayS1.append(f"{dataframe['S1_mass'][index]:.2f} Solar masses")
                mass_arrayS2.append(f"{dataframe['S2_mass'][index]:.2f} Solar masses")
                print('Star1:', dataframe['S1_log_L'][index],dataframe['S1_log_R'][index],
                              'Star2:', dataframe['S2_log_L'][index],dataframe['S2_log_R'][index])
            elif dataframe['state'][index]=='disrupted':
                starstate=f"{dataframe['S1_state'][index]}" + "_" + f"{dataframe['S2_state'][index]}"
                plot_array.append(starstate+string+'_'+'disrupted.jpg') #format: star1_star2_End_disrupted.jpg
                #step_times.append(f"{dataframe['time'][index]}")
                temp_arrayS1.append(Temperature(dataframe['S1_log_L'][index],dataframe['S1_log_R'][index]))
                temp_arrayS2.append(Temperature(dataframe['S2_log_L'][index],dataframe['S2_log_R'][index]))  
                mass_arrayS1.append(f"{dataframe['S1_mass'][index]:.2f} Solar masses")
                mass_arrayS2.append(f"{dataframe['S2_mass'][index]:.2f} Solar masses")
                print('Star1:', dataframe['S1_log_L'][index],dataframe['S1_log_R'][index],
                              'Star2:', dataframe['S2_log_L'][index],dataframe['S2_log_R'][index])
                
        #if (dataframe['S1_mass'][index]>0.6)&(dataframe['S1_mass'][index]<0.8):

                              
        if dataframe['event'][index]=='CC1': #if the function reads CC1 (event)
            starstate=f"{dataframe['S1_state'][index]}" + "_" + f"{dataframe['S2_state'][index]}"
            plot_array.append(starstate+'_'+"CC1.jpg") #format: Star1_star2_CC1.jpg
            step_array.append(f"{dataframe['step_names'][index]}")
            temp_arrayS1.append(Temperature(dataframe['S1_log_L'][index],dataframe['S1_log_R'][index]))
            temp_arrayS2.append(Temperature(dataframe['S2_log_L'][index],dataframe['S2_log_R'][index]))
            mass_arrayS1.append(f"{dataframe['S1_mass'][index]:.2f} Solar masses")
            mass_arrayS2.append(f"{dataframe['S2_mass'][index]:.2f} Solar masses")
            separation_array.append(f"{dataframe['separation'][index]:.2f} Solar Radii")
            print('Star1:', dataframe['S1_log_L'][index],dataframe['S1_log_R'][index],
                              'Star2:', dataframe['S2_log_L'][index],dataframe['S2_log_R'][index])
        
        if dataframe['event'][index]=='oRLOF2': #if the functions reads oRLOF2 state
            starstate=f"{dataframe['S1_state'][index]}" + "_" + f"{dataframe['S2_state'][index]}"
            plot_array.append(starstate+'_'+"oRLOF2.jpg")
            step_array.append(f"{dataframe['step_names'][index]}")
            step_times.append(f"{dataframe['time'][index]/1e6:.0f} Myrs")
            temp_arrayS1.append(Temperature(dataframe['S1_log_L'][index],dataframe['S1_log_R'][index]))
            temp_arrayS2.append(Temperature(dataframe['S2_log_L'][index],dataframe['S2_log_R'][index]))
            mass_arrayS1.append(f"{dataframe['S1_mass'][index]:.2f} Solar masses")
            mass_arrayS2.append(f"{dataframe['S2_mass'][index]:.2f} Solar masses")
            separation_array.append(f"{dataframe['separation'][index]:.2f} Solar Radii")
            print('Star1:', dataframe['S1_log_L'][index],dataframe['S1_log_R'][index],
                              'Star2:', dataframe['S2_log_L'][index],dataframe['S2_log_R'][index])
       
        if dataframe['state'][index]=='RLO2': #if the function reads RLO2 (state)
            starstate=f"{dataframe['S1_state'][index]}" + "_" + f"{dataframe['S2_state'][index]}"
            plot_array.append(starstate+'_'+"RLO2.jpg")
            step_array.append(f"{dataframe['step_names'][index]}")
            step_times.append(f"{dataframe['time'][index]/1e6:.0f} Myrs")
            temp_arrayS1.append(Temperature(dataframe['S1_log_L'][index],dataframe['S1_log_R'][index]))
            temp_arrayS2.append(Temperature(dataframe['S2_log_L'][index],dataframe['S2_log_R'][index]))
            mass_arrayS1.append(f"{dataframe['S1_mass'][index]:.2f} Solar masses")
            mass_arrayS2.append(f"{dataframe['S2_mass'][index]:.2f} Solar masses")
            separation_array.append(f"{dataframe['separation'][index]:.2f} Solar Radii")
            print('Star1:', dataframe['S1_log_L'][index],dataframe['S1_log_R'][index],
                              'Star2:', dataframe['S2_log_L'][index],dataframe['S2_log_R'][index])
        
        #For null columns 
        
        if pd.isnull(dataframe['event'][index]):# if the event column is NaN then it will go to the state column
            print('!')
            starstate=f"{dataframe['S1_state'][index]}" + "_" + f"{dataframe['S2_state'][index]}"
            newname=f"{dataframe['state'][index]}" + "_" + f"{dataframe['step_names'][index]}.jpg"
            step_array.append(f"{dataframe['step_names'][index]}")
            #step_times.append(f"{dataframe['time'][index]}")
            temp_arrayS1.append(Temperature(dataframe['S1_log_L'][index],dataframe['S1_log_R'][index]))
            temp_arrayS2.append(Temperature(dataframe['S2_log_L'][index],dataframe['S2_log_R'][index]))
            mass_arrayS1.append(f"{dataframe['S1_mass'][index]:.2f} Solar masses")
            mass_arrayS2.append(f"{dataframe['S2_mass'][index]:.2f} Solar masses")
            separation_array.append(f"{dataframe['separation'][index]:.2f} Solar Radii")
            print('Star1:', dataframe['S1_log_L'][index],dataframe['S1_log_R'][index],
                              'Star2:', dataframe['S2_log_L'][index],dataframe['S2_log_R'][index])
            print(newname)
            plot_array.append(starstate+newname) #format:Star1_star2_state_stepname
        
        if pd.isnull(dataframe['separation'][index]):#if seperation column is NaN
            separation_array.append(f"{0}")
        
    print(plot_array,temp_arrayS1,temp_arrayS2)
    return plot_array,step_array,step_times,temp_arrayS1,temp_arrayS2, mass_arrayS1, mass_arrayS2, separation_array

                              
                              
            

def vdh(plot_array,step_array,step_times, mass_array_S1, mass_array_S2, separation_array, dataframe): 
    gs1 = gridspec.GridSpec(len(plot_array), 3)
    print(len(plot_array))
    print(plot_array)
    fig = plt.figure(tight_layout=True, figsize=(12*3,12*len(plot_array))) 
    axes_array=[]
    axes2_array=[]                              
    for n,filename in enumerate(plot_array):
        image_path = os.path.join('Visualization_Data', filename)
                              
        if os.path.exists(image_path):

            
            # Open and display the image with istich 
            im = Image.open(image_path)  
            img1 = np.array(im)
            
                        
                                  
            #imgpath = np.asarray(Image.open(image_path))
            #img=cv2.imread(image_path)
            #print(repr(imgpath))

            #img = imgpath[:, :, 0]
            #RGB_img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

            title=filename.split(".")[0].split('_')[-1] #different titles will be placed
            ax = fig.add_subplot(gs1[n, 1])
            ax2 = fig.add_subplot(gs1[n,0])
            ax3= fig.add_subplot(gs1[n,2])
            axes_array.append(ax)  # Add the current ax to the list
            axes2_array.append(ax2)
            ax.set_title(title, fontsize=50) 
            chartobject(ax, img1)
            plt.setp(ax.get_xticklabels(), visible=False)
            plt.setp(ax.get_yticklabels(), visible=False)
            ax.tick_params(labelbottom=False, left=False, labelleft=False, bottom=False)
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)
            ax.spines['left'].set_visible(False)
            ax.spines['bottom'].set_visible(False)
            plt.setp(ax2.get_xticklabels(), visible=False)
            plt.setp(ax2.get_yticklabels(), visible=False)
            ax2.tick_params(labelbottom=False, left=False, labelleft=False, bottom=False)
            ax2.spines['right'].set_visible(False)
            ax2.spines['top'].set_visible(False)
            ax2.spines['left'].set_visible(False)
            ax2.spines['bottom'].set_visible(False)
            plt.setp(ax3.get_xticklabels(), visible=False)
            plt.setp(ax3.get_yticklabels(), visible=False)
            ax3.tick_params(labelbottom=False, left=False, labelleft=False,  bottom=False)
            ax3.spines['right'].set_visible(False)
            ax3.spines['top'].set_visible(False)
            ax3.spines['left'].set_visible(False)
            ax3.spines['bottom'].set_visible(False)
            #ax2.set_xlimit(0,5)
            label = f"{step_times[n]}" # Customize the annotation text as needed
            label2=f"S1: {mass_arrayS1[n]}"
            label3=f"S2: {mass_arrayS2[n]}"
            label4=f"separation: {separation_array[n]}"
            ax2.text(0.1,0.5, label, color='black',size=50,transform=ax2.transAxes)
            ax2.scatter(0.05,0.5, color='black',transform=ax2.transAxes,marker='o', s=300, facecolor='black',linewidth=5)
            ax3.text(1,0.6, label2, color='slategrey', fontsize=40,transform=ax3.transAxes)
            ax3.text(1,0.5, label3, color='indianred', fontsize=40,transform=ax3.transAxes)
            ax3.text(1,0.4, label4, color='black', fontsize=40,transform=ax3.transAxes)                    
            ax3.scatter(0.9,0.6, color='slategrey',transform=ax3.transAxes,marker='*', s=300, facecolor='slategrey',linewidth=5)
            ax3.scatter(0.9,0.5, color='indianred',transform=ax3.transAxes,marker='*', s=300, facecolor='indianred',linewidth=5) 
            ax3.scatter(0.9,0.4, color='black',transform=ax3.transAxes,marker='$S$', s=500, facecolor='black',linewidth=2)
                                          #ha='right', va='center')

            #ax2.scatter()
            
            #ax2 = fig.add_subplot(gs1[2])
            #plt.setp(ax2.get_xticklabels(), visible=False)
            #plt.setp(ax2.get_yticklabels(), visible=False)
            #ax2.tick_params(labelbottom=False, left=False, labelleft=False, bottom=False)
            #ax2.spines['right'].set_visible(False)
            #ax2.spines['top'].set_visible(False)
            #ax2.spines['left'].set_visible(False)
            #ax2.spines['bottom'].set_visible(False)



        else:
            print(f"Image file {filename} not found in the database.")
    rightbar='bar,fraction=-0.3'
    leftbar='bar,fraction=0.3'
    for i,ax in enumerate(axes_array[:-1]):
        if i%2==0:
            xy1=(1,0.5)#for right side
            xy2=(1,0.5)
            bar=rightbar
            rightx=1
            righty=-1
            x=rightx
            y=righty

        else:
            xy1=(0,0.5)#left side
            xy2=(0,0.5)
            bar=leftbar
            leftx=-1
            lefty=-1
            x=leftx
            y=lefty
    
            
        # Adding annotation to the line
        #label = f"{step_array[i]}"  # Customize the annotation text as needed
        #print(label)
        #ax.text(x, y, label, color='slategrey', fontsize=24,
                                          #ha='right', va='center', rotation=90, transform=ax.transAxes)                          
        con1 = ConnectionPatch(xyA=xy1, xyB=xy2, coordsA='axes fraction', coordsB="axes fraction",
                                        axesA=ax, axesB=axes_array[i+1], arrowstyle='->', connectionstyle=bar,
                                        color="slategrey", lw=3, label='orlof')  # <-- Adding "orlof" label here
            # Add the connection patch to the figure
        fig.add_artist(con1)
    
    
        
    timeline = ConnectionPatch(xyA=(0,0.5), xyB=(0,0.5), coordsA='axes fraction', coordsB="axes fraction",
                                    axesA=axes2_array[0], axesB=axes2_array[-1], arrowstyle='->', connectionstyle='arc3,rad=0',
                                    color="black", lw=3, label='orlof')  # <-- Adding "orlof" label here
        # Add the connection patch to the figure
    fig.add_artist(timeline)
                                



# Example usage
plot_array,step_array,step_times,temp_arrayS1,temp_arrayS2, mass_arrayS1, mass_arrayS2, separation_array=getimage(MiniVizView)
vdh(plot_array,step_array,step_times, mass_arrayS1, mass_arrayS2, separation_array, MiniVizView)
print('plot_array',plot_array)
print('step_array',step_array)
print('step_times', step_times)
print('temp_arrayS1',temp_arrayS1)
print('temp_arrayS2',temp_arrayS2)
#flowchart(plot_array)
plt.savefig("Stellar Evolution Charts/ZAMScharttest0live.png",dpi=300, bbox_inches='tight')